In [2]:
# dependencies
# Install core packages
!pip install -q langgraph langchain transformers torch sentence-transformers

# Install Oracle DB driver
!pip install -q oracledb

# Install misc dependencies
!pip install -q python-dotenv uuid requests

import gradio as gr
import uuid
import pandas as pd
# --- Global State Management ---
# We use a dictionary to store the cart state per session ID, allowing multi-user chat.
SESSION_CARTS = {}


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
# Verify If GPU is available
import torch

# Check for GPU availability
gpu_available = torch.cuda.is_available()
print(f"GPU available: {gpu_available}")

if gpu_available:
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: GPU not found. Agent will run on CPU, which may be slow.")

GPU available: False


In [4]:
# Load Environment Variables
from google.colab import userdata
import os

# Load credentials from Colab secrets
os.environ["ORACLE_USER"] = userdata.get('ORACLE_USERNAME')
os.environ["ORACLE_PASSWORD"] = userdata.get('ORACLE_PASSWORD')
os.environ["ORACLE_DSN"] = userdata.get('ORACLE_DSN')
os.environ["ORACLE_WALLET_PASSROD"] = userdata.get('ORACLE_WALLET_PASSWORD')


print("Oracle credentials loaded from Colab secrets.")

Oracle credentials loaded from Colab secrets.


In [5]:
# Model Integration
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

# Model selection - choose a small, fast model
model_name = "google/flan-t5-base"

# Load the model and tokenizer locally
print(f"Loading Hugging Face model: {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto")

# Create a text generation pipeline
generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

print("Hugging Face model loaded successfully.")



Loading Hugging Face model: google/flan-t5-base...


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


Hugging Face model loaded successfully.


In [6]:
# function for intent detection
def get_intent(query: str) -> str:
    """Uses the HF model to classify user intent."""
    prompt = f"Categorize the following user query into one of these intents: 'search_products', 'add_to_cart', 'check_cart', 'place_order', 'general_query'.\n\nQuery: {query}\n\nIntent:"

    # Use the generator pipeline
    result = generator(prompt, max_length=20, do_sample=False)

    # Simple post-processing
    intent = result[0]['generated_text'].strip().lower()

    # Fallback to a rule-based system for robustness
    if "search" in query.lower() or "find" in query.lower() or "show" in query.lower():
        return "search_products"
    elif "add" in query.lower() or "cart" in query.lower():
        return "add_to_cart"
    elif "place" in query.lower() or "order" in query.lower() or "checkout" in query.lower():
        return "place_order"
    else:
        return intent if intent in ['search_products', 'add_to_cart', 'place_order'] else "general_query"

In [7]:
# Oracle 23 Ai Connection

import oracledb
from contextlib import contextmanager

# Function to safely connect to the database
@contextmanager
def get_db_connection():
    """Context manager for a secure database connection."""
    connection = None
    try:
        # Check for wallet file (e.g., in a zipped folder)
            print("Connecting using a secure wallet...")
            connection = oracledb.connect(config_dir="/content/wallet", dsn=os.environ["ORACLE_DSN"],
                                          password=os.environ["ORACLE_PASSWORD"], user=os.environ["ORACLE_USER"],wallet_location="/content/wallet",wallet_password=os.environ["ORACLE_WALLET_PASSROD"])
            yield connection
    except oracledb.Error as e:
        print(f"Database connection error: {e}")
        raise
    finally:
        if connection:
            connection.close()
            print("Database connection closed.")

def execute_sql(sql_statement: str, params: dict = None, fetch: bool = False):
    """Executes a SQL statement and returns results if needed."""
    try:
        with get_db_connection() as conn:
            with conn.cursor() as cursor:
                if params:
                    cursor.execute(sql_statement, params)
                else:
                    cursor.execute(sql_statement)

                if fetch:
                    return cursor.fetchall()
                else:
                    conn.commit()
    except Exception as e:
        print(f"SQL execution error: {e}")
        return None

In [8]:
# Tool Definitions

from typing import List, Dict, Callable
from sentence_transformers import SentenceTransformer
import uuid
import requests
import array # Import array

# Load the sentence-transformer model for embedding
print("Loading sentence-transformers model...")
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cuda' if torch.cuda.is_available() else 'cpu')
print("Embedding model loaded.")

# Central tool registry
TOOL_REGISTRY: Dict[str, Callable] = {}

def register_tool(func: Callable):
    """Decorator to register a tool function."""
    TOOL_REGISTRY[func.__name__] = func
    return func

# --- Required Tools ---

@register_tool
def search_products(query: str) -> List[Dict]:
    """
    Performs a semantic search for products using a vector database.
    Args:
        query (str): The user's search query.
    Returns:
        List[Dict]: A list of dictionaries, each representing a product.
    """
    try:
        # Generate the embedding for the search query
        query_embedding = embedding_model.encode(query, convert_to_tensor=False).tolist() # Convert to list

        # Oracle requires vector data as Python array.array('f') for FLOAT32
        query_vec_bind = array.array('f', query_embedding)


        # SQL for vector search using Oracle 23ai's native functionality
        sql = """
        SELECT product_id, name, description, price
        FROM PRODUCTS
        ORDER BY VECTOR_DISTANCE(embedding, :query_vec)
        FETCH FIRST 5 ROWS ONLY
        """

        with get_db_connection() as conn:
            with conn.cursor() as cursor:
                cursor.execute(sql, query_vec=query_vec_bind) # Use the array.array object
                results = cursor.fetchall()

        # Format the results into a list of dictionaries
        product_list = [
            {"product_id": p[0], "name": p[1], "description": p[2], "price": p[3]}
            for p in results
        ]
        print(f"Found {len(product_list)} products.")
        return product_list
    except Exception as e:
        print(f"Error during product search: {e}")
        return []

@register_tool
def add_to_cart(user_session_id: str, product_id: str) -> Dict:
    """
    Inserts a product into the CART table for a given user session.
    Args:
        user_session_id (str): A unique identifier for the user's session.
        product_id (str): The ID of the product to add.
    Returns:
        Dict: A status dictionary.
    """
    try:
        sql = "INSERT INTO CART (user_session_id, product_id) VALUES (:session_id, :prod_id)"
        execute_sql(sql, params={"session_id": user_session_id, "prod_id": product_id})
        return {"status": "success", "message": f"Product {product_id} added to cart."}
    except Exception as e:
        return {"status": "error", "message": str(e)}

# --- Extensibility Hooks (Stubs) ---

@register_tool
def place_order(user_session_id: str, shipping_address: str) -> Dict:
    """Simulates a call to a REST API to place an order."""
    print(f"Simulating order placement for session {user_session_id} to {shipping_address}...")
    # Example of a simulated API call
    response = requests.post("https://api.example.com/place_order", json={
        "session_id": user_session_id,
        "address": shipping_address
    })
    return {"status": "success", "message": "Order placed successfully."}

@register_tool
def fetch_user_profile(user_id: str) -> Dict:
    """Placeholder for fetching user data from a CRM."""
    print(f"Fetching user profile for user_id {user_id}...")
    return {"user_id": user_id, "name": "John Doe", "email": "john.doe@example.com"}

@register_tool
def check_inventory(product_id: str) -> bool:
    """Placeholder for checking product inventory."""
    print(f"Checking inventory for product_id {product_id}...")
    return True # Assume in stock for this demo

Loading sentence-transformers model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model loaded.


In [9]:
from langgraph.graph import StateGraph, END, START
from typing import TypedDict, Annotated, List, Dict
import operator
import uuid

# --- 5.1 Define Agent State ---
# We use TypedDict for clearer state management, a common practice in LangGraph.

class AgentState(TypedDict):
    """
    Represents the state of the e-commerce agent's conversation.
    """
    user_session_id: str
    user_query: str
    tool_to_run: str  # The determined tool name
    last_tool_output: List[Dict] | Dict | None
    cart_items: List[Dict]
    agent_response: str

# --- 5.2 Define Graph Nodes ---

def user_input_node(state: AgentState):
    """Node for initializing the state with the user's query."""
    print("-> Node: User Input Received")
    # This node primarily receives the external input and passes the full state
    return state

def router_node(state: AgentState):
    """Node to route the query to the correct tool based on intent."""
    query = state["user_query"]
    intent = get_intent(query)
    print(f"-> Node: Intent Detected: {intent}")

    # Update the state with the determined intent/tool
    if intent in TOOL_REGISTRY:
        return {"tool_to_run": intent}
    else:
        # Fallback for general greetings or unknown queries
        return {"tool_to_run": "general_query"}

def tool_execution_node(state: AgentState):
    """Node to execute the selected tool and capture its output."""
    tool_name = state["tool_to_run"]
    tool_func = TOOL_REGISTRY.get(tool_name)
    output = None

    print(f"-> Node: Executing Tool: {tool_name}")

    if tool_name == "search_products":
        # Call the actual product search function with vector DB lookup
        output = tool_func(query=state["user_query"])

    elif tool_name == "add_to_cart":
        # Simplified: In a real agent, you'd parse product_id from the query or prior search context
        # For this demo, we assume the user refers to a known product from the last search (e.g., "JBL-Flip-6")
        product_id = "JBL-Flip-6" # Hardcoded ID for demo continuity
        output = tool_func(user_session_id=state["user_session_id"], product_id=product_id)

        # Update the cart in the state if successful
        if output and output.get("status") == "success":
             state["cart_items"].append({"product_id": product_id, "name": product_id}) # Placeholder name

    elif tool_name == "general_query":
        output = {"status": "info", "message": "I can help you search products or manage your cart. Try 'Show me wireless earbuds'"}

    else:
        # Placeholder for extensibility hooks (place_order, fetch_user_profile, etc.)
        output = tool_func(user_session_id=state["user_session_id"], shipping_address="123 Agent St.")

    return {"last_tool_output": output, "cart_items": state["cart_items"]}

def response_generation_node(state: AgentState):
    """Node to generate a final response to the user based on tool output."""
    print(f"-> Node: Generating Response")

    tool_output = state.get("last_tool_output")
    tool_name = state["tool_to_run"]
    response = ""

    if tool_name == "search_products" and isinstance(tool_output, list):
        if tool_output:
            df = pd.DataFrame(tool_output)
            response = "I found these top products for you:\n"
            response += df[['name', 'price']].to_markdown(index=False, floatfmt=".2f")
            response += "\n\nWhich product would you like to know more about or add to your cart?"
        else:
            response = "Sorry, I couldn't find any products matching that query. Could you try different keywords?"

    elif tool_name == "add_to_cart":
        num_items = len(state.get("cart_items", []))
        response = f"✅ Success! {tool_output.get('message', 'Item added to cart.')}\nYour cart now holds {num_items} item(s)."

    elif tool_name == "general_query":
        response = tool_output.get("message")

    else:
        response = f"Operation complete. Status: {tool_output.get('status')}. Message: {tool_output.get('message', 'Thank you for shopping!')}"

    return {"agent_response": response}

# --- 5.3 Build the LangGraph Workflow ---

# Create a dummy DataFrame import for the response_generation_node to work
import pandas as pd

workflow = StateGraph(AgentState)

# 1. Define Nodes
workflow.add_node("user_input", user_input_node)
workflow.add_node("router", router_node)
workflow.add_node("tool_execution", tool_execution_node)
workflow.add_node("response_generation", response_generation_node)

# 2. Define Entry Point (REQUIRED)
workflow.add_edge(START, "user_input")

# 3. Define the Flow
workflow.add_edge("user_input", "router")
workflow.add_edge("router", "tool_execution")
workflow.add_edge("tool_execution", "response_generation")

# 4. Define Exit Point
workflow.add_edge("response_generation", END)

# Compile the graph
app = workflow.compile()
print("\n✅ LangGraph Agent compiled successfully with entry point: START -> user_input")


✅ LangGraph Agent compiled successfully with entry point: START -> user_input


In [10]:
# App Runner
import pandas as pd
from IPython.display import display, Markdown
import time

# Create a demo user session ID
demo_session_id = uuid.uuid4().hex

def run_agent(query: str, session_id: str, current_cart: List[Dict]):
    """
    Runs a full turn of the LangGraph agent, starting from the START node.
    """

    # 1. Initialize the State for the current turn
    initial_state: AgentState = {
        "user_session_id": session_id,
        "user_query": query,
        "cart_items": current_cart,
        "tool_to_run": "",
        "last_tool_output": None,
        "agent_response": ""
    }

    print(f"\n💬 User: {query}")
    print("-" * 30)

    # Run the graph
    result = app.invoke(initial_state)


    # The final output is the response from the response_generation_node
    response = result

    print(f"🤖 Agent: {response}")

# --- Demo conversation ---

# 1. Setup Session
demo_session_id = uuid.uuid4().hex
user_cart = []
print(f"Starting new session: {demo_session_id}")



# Turn 1: User asks to search for products
run_agent("Show me Bluetooth speakers under $80", session_id=demo_session_id,current_cart=user_cart)
# Expected output: Agent uses search_products tool and lists products from the DB.

print("\n\n" + "-"*50 + "\n\n")

# Turn 2: User asks to add a specific item to the cart
# run_agent("Add the JBL Flip 6 to my cart", session_id=demo_session_id)
# # Expected output: Agent uses add_to_cart tool and confirms the item has been added.

print("\n\n" + "-"*50 + "\n\n")

# Turn 3: User asks a general question
# run_agent("What's a smart speaker?", session_id=demo_session_id)
# Expected output: Agent identifies a general query and gives a canned response.

Starting new session: 1828b39d9b2b4d9794ffffd50e315e14

💬 User: Show me Bluetooth speakers under $80
------------------------------
-> Node: User Input Received


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


-> Node: Intent Detected: search_products
-> Node: Executing Tool: search_products
Connecting using a secure wallet...
Database connection closed.
Found 5 products.
-> Node: Generating Response
🤖 Agent: {'user_session_id': '1828b39d9b2b4d9794ffffd50e315e14', 'user_query': 'Show me Bluetooth speakers under $80', 'tool_to_run': 'search_products', 'last_tool_output': [{'product_id': '82AB89D263', 'name': 'JBL Flip 6 Speaker', 'description': <oracledb.LOB object at 0x7ee6a7b49d30>, 'price': 129.99}, {'product_id': '3F8B66648F', 'name': 'Apple HomePod mini', 'description': <oracledb.LOB object at 0x7ee6a7b49c40>, 'price': 99}, {'product_id': '60D147BD46', 'name': 'Samsung Galaxy S24 Ultra', 'description': <oracledb.LOB object at 0x7ee6a7b49100>, 'price': 1099}, {'product_id': 'E24CE0986E', 'name': 'Google Nest Hub Max', 'description': <oracledb.LOB object at 0x7ee6a7b49580>, 'price': 229}, {'product_id': 'E10CA3F376', 'name': 'Bose QuietComfort Earbuds II', 'description': <oracledb.LOB obje

In [22]:
import uuid
import gradio as gr

SESSION_CARTS = {}

def predict(message: str, history: list, session_id: str):
    global SESSION_CARTS

    # If session_id is empty or invalid, create new session
    if not session_id or session_id not in SESSION_CARTS:
        session_id = uuid.uuid4().hex
        SESSION_CARTS[session_id] = []
        print(f"**NEW SESSION STARTED:** {session_id}")

    current_cart = SESSION_CARTS[session_id]

    initial_state = {
        "user_session_id": session_id,
        "user_query": message,
        "cart_items": current_cart,
        "tool_to_run": "",
        "last_tool_output": None,
        "agent_response": ""
    }

    try:
        final_state = app.invoke(initial_state)  # Replace with real LangGraph call
    except Exception as e:
        agent_response = f"An error occurred: {e}"
        final_state = {**initial_state, 'agent_response': agent_response}

    agent_response = final_state.get('agent_response', "Sorry, I couldn't process that.")
    updated_cart = final_state.get('cart_items', current_cart)
    SESSION_CARTS[session_id] = updated_cart

    # Format cart status as Markdown
    if updated_cart:
        items_md = "\n".join(f"- {item}" for item in updated_cart)
        cart_status = f"**🛒 Cart ({len(updated_cart)} item{'s' if len(updated_cart) != 1 else ''})**\n{items_md}"
    else:
        cart_status = "**🛒 Current Cart (0 items)**"

    return agent_response, session_id, cart_status


# --- UI ---
system_message = "Powered by Oracle 23ai Vector Search + LangGraph"

# Define components
session_state = gr.Textbox(
    label="Session ID",
    value="",  # Start empty; predict will populate
    interactive=False,
    visible=False
)
cart_display = gr.Markdown("**🛒 Current Cart (0 items)**")

with gr.Blocks(title="Oracle 23ai Vector Search Agent") as demo:
    gr.Markdown("# 🛍️ AI Vector Search Agent (Oracle 23ai + LangGraph)")

    with gr.Row():
        with gr.Column(scale=2):
            # ✅ CORRECT FORMAT: list of lists, matching additional_inputs
            examples = [
                ["Show me durable speakers under $150", ""],
                ["Add the JBL Flip 6 to my cart", ""],
                ["Hello there!", ""]
            ]

            chatbot = gr.ChatInterface(
                fn=predict,
                examples=examples,  # Must be list[list] when additional_inputs used
                title="E-commerce Assistant",
                additional_inputs=[session_state],
                additional_outputs=[session_state, cart_display],
                autofocus=True
            )

        with gr.Column(scale=1):
            gr.Markdown("### Agent State")
            cart_display      # No .render()
            gr.Markdown("---")
            gr.Markdown(system_message)
            session_state     # No .render()

if __name__ == "__main__":
    demo.launch(debug=True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:348: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c0fd9623ac14938fe6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**NEW SESSION STARTED:** 3a3a2eca17de44548448f9731ecd0cfb
-> Node: User Input Received


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


-> Node: Intent Detected: add_to_cart
-> Node: Executing Tool: add_to_cart
Connecting using a secure wallet...
Database connection closed.
-> Node: Generating Response
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://bae565e584488ccce0.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://51b07de3b158ccc702.gradio.live
Killing tunnel 127.0.0.1:7863 <> https://c0fd9623ac14938fe6.gradio.live


In [19]:
# Setup Database and insert few products

import oracledb
from sentence_transformers import SentenceTransformer
import array
import uuid
import os
from contextlib import contextmanager

# --- 1. Configuration & Setup ---

# Initialize the Sentence Transformer model
try:
    print("Loading embedding model: all-MiniLM-L6-v2 (384-dim)")
    # Using 'cpu' for a guaranteed start, though Colab often maps to GPU
    embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cpu')
    print("Model loaded.")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# --- Database Connection Context Manager ---


def setup_database():
    """Drops and recreates the PRODUCTS and CART tables."""
    print("\n--- Starting Database Setup (DDL) ---")

    # Use PL/SQL block to conditionally drop tables without erroring if they don't exist
    ddl_script = """
    BEGIN
        EXECUTE IMMEDIATE 'DROP TABLE PRODUCTS CASCADE CONSTRAINTS';
        EXCEPTION WHEN OTHERS THEN IF SQLCODE <> -942 THEN RAISE; END IF;
    END;
    """

    products_ddl = """
    CREATE TABLE PRODUCTS (
      product_id VARCHAR2(50) PRIMARY KEY,
      name VARCHAR2(255),
      description CLOB,
      price NUMBER,
      -- 384 dimensions, 32-bit floating point format
      embedding VECTOR(384, FLOAT32)
    )
    """

    # Dropping CART for completeness, though only PRODUCTS is loaded here
    cart_ddl = """
    CREATE TABLE CART (
      cart_id RAW(16) DEFAULT SYS_GUID(),
      user_session_id VARCHAR2(50),
      product_id VARCHAR2(50),
      added_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """

    try:
        with get_db_connection() as conn:
            with conn.cursor() as cursor:
                # Drop existing table safely
                print("Dropping old PRODUCTS table...")
                try:
                    cursor.execute("DROP TABLE PRODUCTS CASCADE CONSTRAINTS")
                except oracledb.Error as e:
                    if e.args[0].code != 942: # ORA-00942: table or view does not exist
                        raise

                # Create tables
                print("Creating PRODUCTS table with VECTOR column...")
                cursor.execute(products_ddl)

                print("Creating CART table...")
                try:
                    cursor.execute("DROP TABLE CART CASCADE CONSTRAINTS")
                except oracledb.Error as e:
                    if e.args[0].code != 942:
                        raise
                cursor.execute(cart_ddl)

                conn.commit()
                print("✅ Database setup complete.")

    except Exception as e:
        print(f"❌ DDL execution error: {e}")
        raise

# --- 3. Sample Data: 20+ Mobile Devices and Accessories ---

# We'll use the description for embedding to enable semantic search
SAMPLE_PRODUCTS = [
    {"name": "iPhone 15 Pro Max", "description": "Flagship smartphone with A17 Bionic chip, Titanium frame, and a 5x optical zoom camera. The top choice for Apple users.", "price": 1199.00},
    {"name": "Samsung Galaxy S24 Ultra", "description": "Premium Android phone with S Pen, 200MP camera, and a large, dynamic AMOLED display. Best for productivity.", "price": 1099.00},
    {"name": "Google Pixel 8 Pro", "description": "AI-powered phone with the best computational photography and Tensor G3 chip. Pure Android experience.", "price": 999.00},
    {"name": "OnePlus 12", "description": "Fast charging Android phone known for its smooth performance and gaming capabilities. Great value flagship.", "price": 799.00},
    {"name": "Xiaomi 14 Pro", "description": "High-end mobile with a powerful Snapdragon processor and Leica-co-engineered optics.", "price": 850.00},
    {"name": "iPad Pro (M4)", "description": "Ultra-thin tablet with M4 chip, tandem OLED screen, perfect for creative professionals and artists.", "price": 999.00},
    {"name": "Samsung Galaxy Tab S9", "description": "Premium Android tablet with S Pen included, ideal for productivity and entertainment.", "price": 799.00},
    {"name": "Apple Watch Ultra 2", "description": "Rugged smartwatch designed for extreme sports, with enhanced GPS and a bright display. Titanium casing.", "price": 799.00},
    {"name": "Google Pixel Watch 2", "description": "Sleek, circular smartwatch with integrated Fitbit health tracking features. Lightweight and comfortable.", "price": 349.00},
    {"name": "JBL Flip 6 Speaker", "description": "Waterproof portable Bluetooth speaker with bold sound and 12 hours of playtime. Ideal for pool parties.", "price": 129.99},
    {"name": "Bose QuietComfort Earbuds II", "description": "Best-in-class noise cancelling earbuds with a comfortable, secure fit. Perfect for travel or the office.", "price": 279.00},
    {"name": "Anker 737 Power Bank", "description": "24,000mAh power bank with 140W fast charging for laptops and phones. High capacity battery pack.", "price": 149.99},
    {"name": "Spigen Liquid Air Case (iPhone)", "description": "Slim, flexible TPU case with a non-slip grip, providing basic drop protection for iPhone 15.", "price": 19.99},
    {"name": "Samsung Wireless Charger Duo", "description": "Dual wireless charging pad for charging a phone and a watch simultaneously. Sleek black design.", "price": 59.99},
    {"name": "Apple AirTag (4-Pack)", "description": "Small, round tracking device to locate lost items using the Find My network. Precision finding.", "price": 99.00},
    {"name": "Sony WH-1000XM5 Headphones", "description": "Industry-leading noise cancelling over-ear headphones with exceptional sound quality. Comfortable design.", "price": 399.00},
    {"name": "Razer Kishi V2 Controller", "description": "Mobile gaming controller that connects directly to the phone for low-latency play. Universal fit.", "price": 99.99},
    {"name": "UGREEN USB-C Hub 6-in-1", "description": "Multi-port adapter with HDMI, USB-A, and SD card slots for laptops and tablets. Essential accessory.", "price": 45.00},
    {"name": "PopSockets PopGrip", "description": "Collapsible grip and stand that provides a secure hold for texting or taking selfies. Various colors.", "price": 15.00},
    {"name": "Google Nest Hub Max", "description": "Smart display for video calls, controlling smart home devices, and viewing recipes.", "price": 229.00},
    {"name": "Apple HomePod mini", "description": "Compact smart speaker with 360-degree sound and integration with Siri and HomeKit.", "price": 99.00},
    {"name": "Belkin Screen Protector", "description": "Ultra-thin tempered glass screen protector for the latest mobile devices. Easy application.", "price": 25.00},
]

# --- 4. Generate Embeddings and Execute Insertion ---

def insert_products():
    """Generates embeddings and performs the bulk insertion into the PRODUCTS table."""
    insert_data = []
    print(f"\n--- Starting Data Insertion ---")
    print(f"Generating embeddings for {len(SAMPLE_PRODUCTS)} products...")

    descriptions = [p["description"] for p in SAMPLE_PRODUCTS]

    # Generate all embeddings in a single batch for efficiency
    # Returns a NumPy array of shape (22, 384)
    embeddings = embedding_model.encode(descriptions, convert_to_tensor=False)

    if embeddings.shape[1] != 384:
        raise ValueError(f"Embedding model generated {embeddings.shape[1]} dimensions, expected 384.")

    for i, product in enumerate(SAMPLE_PRODUCTS):
        product_id = uuid.uuid4().hex[:10].upper() # Unique ID
        vector_array = embeddings[i]

        # Convert the NumPy array (float32) to Python's array.array('f') type
        # This is the required bind format for FLOAT32 vectors in python-oracledb
        oracle_vector_bind = array.array('f', vector_array.tolist())

        # Prepare the row for executemany
        insert_data.append((
            product_id,
            product["name"],
            product["description"],
            product["price"],
            oracle_vector_bind
        ))

    print(f"Prepared {len(insert_data)} records for insertion using executemany.")

    sql_insert = """
    INSERT INTO PRODUCTS (product_id, name, description, price, embedding)
    VALUES (:1, :2, :3, :4, :5)
    """

    try:
        with get_db_connection() as connection:
            with connection.cursor() as cursor:
                # Execute the batch insert
                cursor.executemany(sql_insert, insert_data)
                connection.commit()
                print(f"✅ Successfully inserted {len(insert_data)} records with vector embeddings.")

                # Verification Query
                cursor.execute("SELECT COUNT(*) FROM PRODUCTS")
                count = cursor.fetchone()[0]
                print(f"Verification: Total records in PRODUCTS table: {count}")

    except oracledb.Error as e:
        error_obj, = e.args
        print(f"\n❌ Oracle Database Error during insertion: {error_obj.code} - {error_obj.message}")
        print("Check if the table was created successfully and the column types match.")

# --- Execution ---

if __name__ == "__main__":
    try:
        setup_database()
        insert_products()
    except Exception as e:
        print(f"\n🛑 Fatal error during script execution: {e}")

Loading embedding model: all-MiniLM-L6-v2 (384-dim)
Model loaded.

--- Starting Database Setup (DDL) ---
Connecting using a secure wallet...
Dropping old PRODUCTS table...
Creating PRODUCTS table with VECTOR column...
Creating CART table...
✅ Database setup complete.
Database connection closed.

--- Starting Data Insertion ---
Generating embeddings for 22 products...
Prepared 22 records for insertion using executemany.
Connecting using a secure wallet...
✅ Successfully inserted 22 records with vector embeddings.
Verification: Total records in PRODUCTS table: 22
Database connection closed.
